# Metrics

Example: a metric data structure
```py
(9, # 9th epoch's training results for example
 {'metrics/mAP50(B)': 0.3526,
  'metrics/mAP50-95(B)': 0.3526,
  'metrics/precision(B)': 0.01663,
  'metrics/recall(B)': 1.0,
  'val/box_loss': 0.12215,
  'val/cls_loss': 3.02035,
  'val/dfl_loss': 0.14519}
)
```

# Packages

## pip install

In [5]:
%pip install ipython-autotime
%pip install plotly
%pip install -U kaleido
%load_ext autotime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.9 MB/s eta 0:00:00
time: 370 µs (started: 2024-12-12 00:50:07 +00:00)


## import

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from pprint import pprint
from IPython.display import Image
from IPython import display

display.clear_output()

time: 4.24 ms (started: 2024-12-12 00:50:07 +00:00)


## utilities

In [ ]:
import os
import json
from datetime import datetime


class FileManager:
    """
    A class for managing file operations, including creating directories,
    exporting data to JSON files, and importing data from JSON files.
    """

    @staticmethod
    def current_dump_filepath(folder: str) -> str:
        """
        Generates a filepath for a JSON dump file with a timestamp.

        Args:
            folder (str): The folder path to store the dump file.

        Returns:
            str: The generated filepath.
        """

        timestamp = datetime.now().strftime("%Y-%m-%d-%H")
        filename = f"named_train_list-{timestamp}.json"
        return os.path.join(folder, filename)

    @staticmethod
    def export_dump_file(training_data: list, filepath: str) -> None:
        """
        Exports a list of data to a JSON file.

        Args:
            training_data (list): The data to be exported.
            filepath (str): The desired filepath.
        """
        print(f"{filepath=}")

        try:
            os.makedirs(os.path.dirname(filepath), exist_ok=True)
            with open(filepath, "w") as json_file:
                json.dump(training_data, json_file, indent=4)
            print(f"Successfully dump data to {filepath}.")
        except Exception as e:
            print(f"Error during dump to {filepath}: {e}")

    @staticmethod
    def import_dump_file(filepath: str) -> list:
        """
        Imports data from a JSON file.

        Args:
            filepath (str): The filepath of the JSON file.

        Returns:
            list: The imported data as a list.

        Raises:
            FileNotFoundError: If the specified file is not found.
            json.JSONDecodeError: If there's an error decoding the JSON data.
        """

        if filepath is None:
            raise ValueError("filepath argument is required.")

        try:
            with open(filepath, "r") as json_file:
                data = json.load(json_file)
            print(f"Successfully loaded data from {filepath}.")
            return data
        except FileNotFoundError:
            print(f"File {filepath} not found.")
        except json.JSONDecodeError:
            print(f"Error decoding JSON from the file {filepath}.")
        except Exception as e:
            print(f"Error during loading {filepath}: {e}")
        return []


time: 2.16 ms (started: 2024-12-12 01:38:02 +00:00)


## mount drive

# Configuration

In [7]:
from google.colab import drive

drive.mount('/content/drive')
%cd /content
!ln -s "/content/drive/MyDrive/Colab Notebooks/" /gdrive

# local -> gdrive
# !cp /content/mvtecad.zip /gdrive

# gdrive -> local
# !cp /gdrive/yolo/mvtecad.zip /content

Mounted at /content/drive
/content
time: 22.4 s (started: 2024-12-12 00:50:07 +00:00)


## dataset_dict

In [ ]:
import os

# Paths
# dataset_name = "bottle"

dataset_dict = dict(
    name            = 'bottle', # MVTec AD sub-dataset
    base_path       = f'datasets/mvtecad/bottle',
    src_train_path  = f'datasets/mvtecad/bottle/train', # train set copied from test
    src_test_path   = f'datasets/mvtecad/bottle/test', # val set copied from test
    dest_path       = f"datasets/bottle",
    dest_path_aug   = f"datasets/bottle_aug", # augemtation
    data_yaml_path  = f"datasets/bottle/data",
    dump_save_dir   = '/content/drive/MyDrive/Colab Notebooks/yolo/runs',
    dump_plot_dir   = '/content/drive/MyDrive/Colab Notebooks/yolo/plots',
    dump_metrics_dir= f'/content/drive/MyDrive/Colab Notebooks/yolo/metrics',
)
# os.makedirs(f"{dump_metrics_dir}", exist_ok=True)

# Map folder names to class IDs
class_map = dict(
    good          = 0,
    broken_large  = 1,
    broken_small  = 2,
    contamination = 3
)

time: 7.83 ms (started: 2024-12-12 00:50:30 +00:00)


## named_train_list

In [ ]:
import itertools

DEFAULT_PARAMS = dict(
    model_names     = ["yolo11n.pt", 'yolo11s.pt', 'yolo11m.pt'],
    imgsizes        = [256, 512],
    optimizers      = ["SGD", "AdamW", "Adam"],
    learning_rates  = [0.01, 0.005, 0.001],
    batch_size      = 16, # Adjust this if OOM
    epochs          = 50,
    image_augments  = [False, True]
)


def generate_params_list(params: dict=DEFAULT_PARAMS) -> list:
    """Generate a list of parameter combinations."""

    return list(itertools.product(
        params["model_names"],
        params["img_sizes"],
        params["optimizers"],
        params["image_augments"],
    ))


def create_named_train_list(
    params_list: list,
    default_params: dict=DEFAULT_PARAMS
) -> list:
    """Create a list of named training configurations."""

    records = []
    for i, params in enumerate(params_list):
        p_model_name, p_image_size, p_optimizer, p_aug = params
        param_name = '_'.join([
            f"Run{i+1}",
            f"{p_model_name.split('.')[0]}",
            f"{p_image_size}",
            f"{p_optimizer}"
        ]) + ('_Aug' if p_aug else '')

        train_config = {
            "name": param_name,
            "augmented": p_aug,
            "model": p_model_name,
            "img_size": p_image_size,
            "optimizer": p_optimizer,
            "learning_rate": default_params["learning_rates"][0],
            "batch_size": default_params["batch_size"],
            "epochs": default_params["epochs"],
            "data_yaml_path": "",
            "save_dir": "",
            "result": None,
            "model_metrics": None,
            "completed": False,
            "errors": [],
        }
        records.append((param_name, train_config))

    return records


# Start preparing training list
params_list = generate_params_list()
named_train_list = create_named_train_list(params_list)

for name, data in named_train_list:
    print(f"{name}: {data}")

Run1_yolo11n_256_SGD: {'name': 'Run1_yolo11n_256_SGD', 'augmented': False, 'model': 'yolo11n.pt', 'img_size': 256, 'optimizer': 'SGD', 'learning_rate': 0.01, 'batch_size': 16, 'epochs': 50, 'data_yaml_path': '', 'save_dir': '', 'result': None, 'model_metrics': None, 'completed': False, 'errors': []}
Run2_yolo11n_256_SGD_Aug: {'name': 'Run2_yolo11n_256_SGD_Aug', 'augmented': True, 'model': 'yolo11n.pt', 'img_size': 256, 'optimizer': 'SGD', 'learning_rate': 0.01, 'batch_size': 16, 'epochs': 50, 'data_yaml_path': '', 'save_dir': '', 'result': None, 'model_metrics': None, 'completed': False, 'errors': []}
Run3_yolo11n_256_AdamW: {'name': 'Run3_yolo11n_256_AdamW', 'augmented': False, 'model': 'yolo11n.pt', 'img_size': 256, 'optimizer': 'AdamW', 'learning_rate': 0.01, 'batch_size': 16, 'epochs': 50, 'data_yaml_path': '', 'save_dir': '', 'result': None, 'model_metrics': None, 'completed': False, 'errors': []}
Run4_yolo11n_256_AdamW_Aug: {'name': 'Run4_yolo11n_256_AdamW_Aug', 'augmented': True

## import_dump_file

In [ ]:
# test import_dump_file
# prev_filepath = '/content/drive/MyDrive/Colab Notebooks/yolo/metrics/named_train_list-2024-12-11-14.json'
prev_filepath = '/content/combined_train_list.json'
if False:
    data = []
    if False:
        data = import_dump_file(filepath=prev_filepath)
        print('length loaded:', len(data))

    if False and data:
        named_train_list = data
        for name, data in named_train_list:
            print(f"{name}: {data}")
            break
        # local backup and review
        tmp_filepath = '/content/named_train_list.prev.json'
        export_dump_file(training_data=named_train_list,
                         filepath=tmp_filepath)

Successfully loaded data from /content/combined_train_list.json.
length loaded: 36
Run1_yolo11n_256_SGD: {'name': 'Run1_yolo11n_256_SGD', 'augmented': False, 'model': 'yolo11n.pt', 'img_size': 256, 'optimizer': 'SGD', 'learning_rate': 0.01, 'batch_size': 16, 'epochs': 50, 'data_yaml_path': '/content/datasets/bottle/data/Run1_yolo11n_256_SGD_data.yaml', 'save_dir': '/content/drive/MyDrive/Colab Notebooks/yolo/runs', 'result': {'results_dict': {'metrics/precision(B)': 0.8823000972249608, 'metrics/recall(B)': 0.9162490732162863, 'metrics/mAP50(B)': 0.9323916157372041, 'metrics/mAP50-95(B)': 0.932391615737204, 'fitness': 0.9323916157372041}}, 'model_metrics': {'epoch_count': 50, 'epoch_ends': [[0, {'metrics/precision(B)': 0, 'metrics/recall(B)': 0, 'metrics/mAP50(B)': 0, 'metrics/mAP50-95(B)': 0, 'val/box_loss': 0, 'val/cls_loss': 0, 'val/dfl_loss': 0}], [1, {'metrics/precision(B)': 0.03138, 'metrics/recall(B)': 1.0, 'metrics/mAP50(B)': 0.45244, 'metrics/mAP50-95(B)': 0.45244, 'val/box_los

# Customize Callback Function

## EpochBase

In [52]:
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba

class EpochBase():
    def __init__(self, state_data):
        super().__init__()
        self.state_data = state_data # named_train_list

    @staticmethod
    def generate_extended_colors(num_colors: int):
        """
        Generate a large palette of visually distinct colors by combining multiple colormaps.
        """
        cmap1 = plt.get_cmap("tab10")  # High contrast
        cmap2 = plt.get_cmap("Set3")   # Soft but distinct
        cmap3 = plt.get_cmap("Dark2")  # Dark tones
        cmaps = [cmap1, cmap2, cmap3]

        # Generate enough colors by cycling through the colormaps
        colors = []
        for i in range(num_colors):
            cmap = cmaps[i % len(cmaps)]  # Cycle through colormaps
            idx = (i // len(cmaps)) % cmap.N  # Avoid direct repetition
            r, g, b, a = cmap(idx / (cmap.N - 1))
            colors.append(f"rgba({int(r*255)}, {int(g*255)}, {int(b*255)}, {a})")
        return colors

    @staticmethod
    def export_plotly_image(plotly_fig, filepath: str, width=1920, height=1080, scale=3):
        """
        Prefer to export high definition image for post-analysis.
        """
        try:
            os.makedirs(os.path.dirname(filepath), exist_ok=True)
            plotly_fig.write_image(filepath, width=width, height=height, scale=scale)
            print(f"Successfully export image to {filepath}.")
        except Exception as e:
            print(f"Error during exporting image: {e}")

    def get_and_init_state_record(self, model_id):
        """
        Get or initialize the state record for the given model_id.
        """
        print('Lookup model_id:', model_id)
        record = [data for name, data in self.state_data if name == model_id][0]
        if not record.get('model_metrics'):
            record['model_metrics'] = dict(
                epoch_count = 0,
                epoch_ends = [], # trainer.metrics
                epochs = [],
                mAP50_B = [],
                cls_loss = [],
            )

        return record, record['model_metrics']

    def get_total_epochs_from_record(self):
        """
        Get the total number of epochs from all the records.
        """
        return sum([data['model_metrics'].get('epoch_count', 0)
                    for _, data in self.state_data if data['model_metrics']])


time: 1.81 ms (started: 2024-12-12 01:38:16 +00:00)


## EpochDrawing

In [47]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import clear_output

class EpochDrawing(EpochBase):
    """
    Generate a large palette of visually distinct colors by combining multiple colormaps.
    """
    def __init__(self, state_data):
        super().__init__(state_data)
        ### Start generate colors
        self.plotly_colors = self.generate_extended_colors(num_colors=len(state_data))

    def __call__(self, trainer):
        """
        As reference how to use this callback function during training.
        """
        epoch = trainer.epoch
        model_id = trainer.model.args.name  # Unique identifier for the model
        metrics = trainer.metrics
        record, model_metrics = self.get_and_init_state_record(model_id)
        model_metrics['epoch_ends'].append((epoch, metrics))
        model_metrics['epoch_count'] += 1
        total_epochs = self.get_total_epochs_from_record()

        mAP50_B = metrics.get('metrics/mAP50(B)', 0)
        cls_loss = metrics.get('val/cls_loss', 0)

        model_metrics['epochs'].append(model_metrics['epoch_count'])
        model_metrics['mAP50_B'].append(mAP50_B)
        model_metrics['cls_loss'].append(cls_loss)

        # Plot the metrics
        self.plotly_metrics()

    def plotly_metrics(self, is_show: bool=True, is_save: bool=False):
        """
        Major drawing function.
        """
        clear_output(wait=True)  # Clear the previous plot

        # Chart 1: mAP50 over epochs
        fig_map50 = go.Figure()
        total_epochs = self.get_total_epochs_from_record()

        for idx, (model_id, record) in enumerate(self.state_data):
            data=record['model_metrics']
            if not data:
                continue
            legend = f'{model_id} - mAP50'
            color = self.plotly_colors[idx]
            fig_map50.add_trace(
                go.Scatter(
                    x=data['epochs'],
                    y=data['mAP50_B'],
                    mode='lines+markers',
                    line=dict(width=1.0, color=color),
                    marker=dict(size=5, color=color),
                    name=legend,
                    hoverinfo='text',
                    text=[f'{model_id}<br>Epoch: {x}<br>mAP50: {y:.2f}<br>cls_loss: {loss:.2f}'
                          for x, y, loss in zip(data['epochs'], data['mAP50_B'], data['cls_loss'])],
                )
            )

        fig_map50.update_layout(
            title="mAP50 over Epochs for Multiple Models",
            xaxis=dict(title="Epoch", tickmode="linear"),
            yaxis=dict(title="mAP50",
                       range=[0, 1.0]),
            legend=dict(title="Models",
                        orientation="v",
                        yanchor="top", y=1,
                        xanchor="left", x=1.02,
                       ),
            showlegend=True,
        )

        # Chart 2: Classification loss over epochs
        fig_cls_loss = go.Figure()
        for idx, (model_id, record) in enumerate(self.state_data):
            data = record['model_metrics']
            if not data:
                continue
            legend = f'{model_id} - cls_loss'
            color = self.plotly_colors[idx]
            fig_cls_loss.add_trace(
                go.Scatter(
                    x=data['epochs'],
                    y=data['cls_loss'],
                    mode='lines+markers',
                    line=dict(width=1.0, color=color),
                    marker=dict(size=5, color=color),
                    name=legend,
                    hoverinfo='text',
                    text=[f'{model_id}<br>Epoch: {x}<br>Loss: {y:.2f}'
                          for x, y in zip(data['epochs'], data['cls_loss'])],
                )
            )

        fig_cls_loss.update_layout(
            title="Classification Loss over Epochs for Multiple Models",
            xaxis=dict(title="Epoch", tickmode="linear"),
            yaxis=dict(title="Loss",
                       range=[0, 1.5]), # Adjusted for better visibility
            legend=dict(title="Models",
                        orientation="v",
                        yanchor="top", y=1,
                        xanchor="left", x=1.02,
                       ),
            showlegend=True,
        )

        # Display both charts
        if is_show:
            fig_map50.show()
            fig_cls_loss.show()

        # Export high resolution charts
        if is_save:
            self.export_plotly_image(plotly_fig=fig_map50,
                                     filepath=f'bottle_mAP50_plot_{total_epochs}.png')
            self.export_plotly_image(plotly_fig=fig_cls_loss,
                                     filepath=f'bottle_cls_loss_plot_{total_epochs}.png')



time: 2.22 ms (started: 2024-12-12 01:36:43 +00:00)


# Post Analysis

## Load json

In [ ]:
def combine_train_files() -> list:
    # replace with your path (here example we have 3 sources)
    json1_12 = FileManager.import_dump_file(filepath='/content/named_train_list-2024-12-11_Run1-12.json')
    json13_24 = FileManager.import_dump_file(filepath='/content/named_train_list-2024-12-11_Run13-24.json')
    json25_36 = FileManager.import_dump_file(filepath='/content/named_train_list-2024-12-11_Run25-36.json')

    merging_list = [
        (json1_12, range(1, 13)),
        (json13_24, range(13, 25)),
        (json25_36, range(25, 37)),
    ]

    combined_list = []
    for state_data, ranges in merging_list:
        for run in ranges:
            run_name = f'Run{run}'
            data = [data for name, data in state_data if name.split('_')[0] == run_name][0]
            combined_list.append((data['name'], data))
    return combined_list

time: 1.23 ms (started: 2024-12-12 01:44:22 +00:00)


In [60]:
combined_train_list = combine_train_files()

len(combined_train_list)
print(combined_train_list[0])

Successfully loaded data from /content/named_train_list-2024-12-11_Run1-12.json.
Successfully loaded data from /content/named_train_list-2024-12-11_Run13-24.json.
Successfully loaded data from /content/named_train_list-2024-12-11_Run25-36.json.
('Run1_yolo11n_256_SGD', {'name': 'Run1_yolo11n_256_SGD', 'augmented': False, 'model': 'yolo11n.pt', 'img_size': 256, 'optimizer': 'SGD', 'learning_rate': 0.01, 'batch_size': 16, 'epochs': 50, 'data_yaml_path': '/content/datasets/bottle/data/Run1_yolo11n_256_SGD_data.yaml', 'save_dir': '/content/drive/MyDrive/Colab Notebooks/yolo/runs', 'result': {'results_dict': {'metrics/precision(B)': 0.8823000972249608, 'metrics/recall(B)': 0.9162490732162863, 'metrics/mAP50(B)': 0.9323916157372041, 'metrics/mAP50-95(B)': 0.932391615737204, 'fitness': 0.9323916157372041}}, 'model_metrics': {'epoch_count': 50, 'epoch_ends': [[0, {'metrics/precision(B)': 0, 'metrics/recall(B)': 0, 'metrics/mAP50(B)': 0, 'metrics/mAP50-95(B)': 0, 'val/box_loss': 0, 'val/cls_los

In [ ]:
if False:
    export_dump_file(training_data=combined_train_list,
                    filepath='/content/drive/MyDrive/Colab Notebooks/yolo/metrics/combined_train_list.json')

filepath='/content/drive/MyDrive/Colab Notebooks/yolo/metrics/combined_train_list.json'
Successfully dump data to /content/drive/MyDrive/Colab Notebooks/yolo/metrics/combined_train_list.json.
time: 77.8 ms (started: 2024-12-11 17:27:17 +00:00)


## Plotly

In [48]:
# poly_draw = EpochEndCallback(state_data=named_train_list)
poly_draw = EpochDrawing(state_data=combined_train_list)

time: 2.73 ms (started: 2024-12-12 01:36:45 +00:00)


In [49]:
poly_draw.plotly_metrics()

time: 145 ms (started: 2024-12-12 01:36:46 +00:00)


## Save plotly

In [50]:
poly_draw.plotly_metrics(is_show=False, is_save=True)

Successfully export image to bottle_mAP50_plot_1800.png.
Successfully export image to bottle_cls_loss_plot_1800.png.
time: 7.2 s (started: 2024-12-12 01:36:48 +00:00)


In [54]:
model_id, data = combined_train_list[0] # named_train_list[0]

model_metrics = data['model_metrics']
print('last epoch metrics', model_metrics['epoch_ends'][-1])

results_dict = data['result']['results_dict']
print('results_dict:', results_dict)

for k, v in results_dict.items():
    print(k, f'{v:.4f}')


last epoch metrics [49, {'metrics/precision(B)': 0.90761, 'metrics/recall(B)': 0.9044, 'metrics/mAP50(B)': 0.90427, 'metrics/mAP50-95(B)': 0.90096, 'val/box_loss': 0.19886, 'val/cls_loss': 0.536, 'val/dfl_loss': 0.11303}]
results_dict: {'metrics/precision(B)': 0.8823000972249608, 'metrics/recall(B)': 0.9162490732162863, 'metrics/mAP50(B)': 0.9323916157372041, 'metrics/mAP50-95(B)': 0.932391615737204, 'fitness': 0.9323916157372041}
metrics/precision(B) 0.8823
metrics/recall(B) 0.9162
metrics/mAP50(B) 0.9324
metrics/mAP50-95(B) 0.9324
fitness 0.9324
time: 6.79 ms (started: 2024-12-12 01:40:07 +00:00)
